## Assignment with MI
https://developers.google.com/optimization/assignment/assignment_example?hl=zh-cn

In [3]:
!pip install ortools

Looking in indexes: https://pypi.doubanio.com/simple
     |████████████████████████████████| 16.3 MB 611 kB/s eta 0:00:011
     |████████████████████████████████| 126 kB 7.3 MB/s eta 0:00:01
     |████████████████████████████████| 302 kB 4.5 MB/s eta 0:00:01


In [20]:
!export http_proxy=http://10.10.143.227:8889/; export https_proxy=http://10.10.143.227:8889/; export HTTPS_PROXY=http://10.10.143.227:8889/; export HTTP_PROXY=http://10.10.143.227:8889/
! curl cip.cc

IP	: 58.246.115.34
地址	: 中国  上海
运营商	: 联通

数据二	: 上海市 | 联通

数据三	: 中国上海上海市 | 联通

URL	: http://www.cip.cc/58.246.115.34


In [21]:
from ortools.linear_solver import pywraplp


In [26]:
from ortools.linear_solver import pywraplp


def main():
    # Data
    costs = [
        [90, 80, 75, 70],
        [35, 85, 55, 65],
        [125, 95, 90, 95],
        [45, 110, 95, 115],
        [50, 100, 90, 100],
    ]
    num_workers = len(costs)
    num_tasks = len(costs[0])

    # Solver
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')

    if not solver:
        return

    # Variables
    # x[i, j] is an array of 0-1 variables, which will be 1
    # if worker i is assigned to task j.
    x = {}
    for i in range(num_workers):
        for j in range(num_tasks):
            x[i, j] = solver.IntVar(0, 1, '')

    # Constraints
    # Each worker is assigned to at most 1 task.
    for i in range(num_workers):
        solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

    # Each task is assigned to exactly one worker.
    for j in range(num_tasks):
        solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)

    # Objective
    objective_terms = []
    for i in range(num_workers):
        for j in range(num_tasks):
            objective_terms.append(costs[i][j] * x[i, j])
    solver.Minimize(solver.Sum(objective_terms))

    # Solve
    status = solver.Solve()

    # Print solution.
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print(f'Total cost = {solver.Objective().Value()}\n')
        for i in range(num_workers):
            for j in range(num_tasks):
                # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
                if x[i, j].solution_value() > 0.5:
                    print(f'Worker {i} assigned to task {j}.' +
                          f' Cost: {costs[i][j]}')
    else:
        print('No solution found.')


if __name__ == '__main__':
    main()

Total cost = 265.0

Worker 0 assigned to task 3. Cost: 70
Worker 1 assigned to task 2. Cost: 55
Worker 2 assigned to task 1. Cost: 95
Worker 3 assigned to task 0. Cost: 45


 ## CP SAT
OR-Tools中的CP-SAT求解器是一种针对约束编程（Constraint Programming）问题的求解器，其目标是在满足给定约束条件下寻找最优解。与之相比，线性规划（LP）问题是一种线性函数在限定条件下的最优化问题。

CP-SAT求解器可以处理的问题范围更广泛，包括整数规划、布尔规划、排列问题等等，并且具有许多高级特性，如自定义搜索和剪枝策略等等。同时，由于CP-SAT求解器采用的是离散域上的算法，而非LP中的连续域优化方法，因此其求解速度可能更快，对于大规模问题表现得也更佳。

与之相比，LP问题一般情况下需要使用基于单纯形算法的线性规划求解器（如Gurobi、CPLEX等）进行求解，并且只能处理线性约束问题。然而，在某些情况下，尤其是在约束条件为线性问题时，LP求解器可能会更加高效。

综上所述，CP-SAT求解器和LP问题之间主要区别如下：



In [29]:
from ortools.sat.python import cp_model


def main():
    # Data
    costs = [
        [90, 80, 75, 70],
        [35, 85, 55, 65],
        [125, 95, 90, 95],
        [45, 110, 95, 115],
        [50, 100, 90, 100],
    ]
    num_workers = len(costs)
    num_tasks = len(costs[0])

    # Model
    model = cp_model.CpModel()

    # Variables
    x = []
    for i in range(num_workers):
        t = []
        for j in range(num_tasks):
            t.append(model.NewBoolVar(f'x[{i},{j}]'))
        x.append(t)

    # Constraints
    # Each worker is assigned to at most one task.
    for i in range(num_workers):
        model.AddAtMostOne(x[i][j] for j in range(num_tasks))

    # Each task is assigned to exactly one worker.
    for j in range(num_tasks):
        model.AddExactlyOne(x[i][j] for i in range(num_workers))

    # Objective
    objective_terms = []
    for i in range(num_workers):
        for j in range(num_tasks):
            objective_terms.append(costs[i][j] * x[i][j])
    model.Minimize(sum(objective_terms))

    # Solve
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # Print solution.
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print(f'Total cost = {solver.ObjectiveValue()}')
        print()
        for i in range(num_workers):
            for j in range(num_tasks):
                if solver.BooleanValue(x[i][j]):
                    print(
                        f'Worker {i} assigned to task {j} Cost = {costs[i][j]}')
    else:
        print('No solution found.')


if __name__ == '__main__':
    main()

Total cost = 265.0

Worker 0 assigned to task 3 Cost = 70
Worker 1 assigned to task 2 Cost = 55
Worker 2 assigned to task 1 Cost = 95
Worker 3 assigned to task 0 Cost = 45
